In [1]:
from caveclient import CAVEclient
import csv
import vtk
import pandas as pd
import matplotlib as plt
import numpy as np
from meshparty import trimesh_io, trimesh_vtk, mesh_filters
client = CAVEclient('minnie65_phase3_v1')
from ast import literal_eval
from datetime import datetime, timezone
settime = datetime(2023, 9, 19, 4, 0, 0, 0, tzinfo=timezone.utc)

print(vtk.__version__)


9.2.2


In [2]:
df_ids_pred_met = pd.read_csv(r"C:\Users\clare.gamlin\Documents\CG_EM_paper\Prediction_csvs\Final_EM_MetType_Preds_RFC_Resamp_AllFeats_NoRadius.csv")
df_ids_pred_met = df_ids_pred_met.loc[df_ids_pred_met['probability'] >0.54]

##predictions for curated SSTs
cur_ssts = client.materialize.query_table('gamlin_2023_mcs', timestamp=settime)
cur_ssts_pred = pd.merge(df_ids_pred_met, cur_ssts, left_on = 'id', right_on = 'pt_root_id')
pre_id_mc = cur_ssts_pred["id_x"]
print (pre_id_mc.head(15))
##big batch


Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


0     864691135013417622
1     864691135118298333
2     864691135341516741
3     864691135404765166
4     864691135544588584
5     864691135572082925
6     864691135577202181
7     864691135697251738
8     864691135699487522
9     864691135785592004
10    864691135969447781
11    864691136116457636
12    864691136118647832
13    864691136195284556
14    864691136238652476
Name: id_x, dtype: int64


In [3]:
df2 = client.materialize.synapse_query(pre_ids= pre_id_mc, timestamp = settime)

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [4]:

md3 = client.materialize.query_table('aibs_metamodel_celltypes_v661', timestamp = settime) ##Leila's new model as of 12/23/23
soma_counts = md3['pt_root_id'].value_counts().rename_axis('pt_root_id').reset_index(name ='no_of_somas')
dfm = pd.merge(df2, md3, left_on = 'post_pt_root_id', right_on ='pt_root_id', how = 'right')
singlesomas = soma_counts.loc[soma_counts['no_of_somas'] == 1]
single_soma_merge = pd.merge(md3, singlesomas, left_on = 'pt_root_id', right_on = 'pt_root_id', how = 'inner')
dfm_somas = pd.merge(df2, single_soma_merge, left_on = 'post_pt_root_id', right_on = 'pt_root_id', how = 'inner')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('BC','INH')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('MC','INH')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('NGC','INH')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('BPC','INH')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('6P-CT','6P')
dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('6P-IT','6P')
# dfm_somas['cell_type'] = dfm_somas['cell_type'].str.replace('5P-PT','5P-ET')


Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [6]:
mm = trimesh_io.MeshMeta(cv_path=client.info.segmentation_source(),
                         disk_cache_path='minnie_meshes')


In [7]:
myelin_df = pd.read_csv("./mye_annot_ids.csv")
myelin_df_sub = myelin_df[["Coordinate_1", "Coordinate_2", "mesh_id", "met_type"]].copy()
myelin_df_sub['Coordinate_1'] = myelin_df_sub.Coordinate_1.apply(lambda x: literal_eval(str(x)))
myelin_df_sub['Coordinate_2'] = myelin_df_sub.Coordinate_2.apply(lambda x: literal_eval(str(x)))
myelin_df_sub['mesh_id_full'] = myelin_df_sub['mesh_id'].str.rstrip(',')

myelin_df_sub = myelin_df_sub.astype({'mesh_id_full': 'int64'})


In [7]:
# #### just morphology
# for i in pre_id_mc:
#     print (i)
#     mesh = mm.mesh(seg_id = i)
#     prema = trimesh_vtk.mesh_actor(mesh, color =(0.3,0.3,0.3), opacity= 1)
#     prema.GetProperty().SetRepresentationToWireframe()
#     prema.GetProperty().SetLineWidth(1)
    
#     camera = vtk.vtkCamera()
#     view = camera.ParallelProjectionOn()    
#     trimesh_vtk.render_actors([prema],
#                             filename = "{}_morph_only_scale.png".format(i), do_save = False, camera = view)

In [8]:
pre_id_mc = [864691135013417622]
for i in pre_id_mc:
    print (i)
    dfm_i = dfm_somas.loc[dfm_somas['pre_pt_root_id'] == i]
    mesh = mm.mesh(seg_id = i)
    prema = trimesh_vtk.mesh_actor(mesh, color =(0.3,0.3,0.3), opacity= 1)
    prema.GetProperty().SetRepresentationToWireframe()
    prema.GetProperty().SetLineWidth(1)
    
#     syn_loc = np.vstack(dfm_i['ctr_pt_position'])*[4,4,40]
#     sa = trimesh_vtk.point_cloud_actor(syn_loc, size = 2500, color = (0,1,1), opacity = 1)
#     #red
#     L23_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("23P")]
#     L23_loc = np.vstack(L23_syn['ctr_pt_position'])*[4,4,40]
#     L23_actor = trimesh_vtk.point_cloud_actor(L23_loc, size =6000, color = (1, 0.2, 0.2), opacity = 1)
# #     L23_actor = trimesh_vtk.point_cloud_actor(L23_loc, size =6000, color = (0.77, 0.17, 0.24), opacity = 1)

#     #green
#     L4_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("4P")]
#     L4_loc = np.vstack(L4_syn['ctr_pt_position'])*[4,4,40]
#     L4_actor = trimesh_vtk.point_cloud_actor(L4_loc, size = 6000, color = (0, 1, 0), opacity = 1)
# #     L4_actor = trimesh_vtk.point_cloud_actor(L4_loc, size = 6000, color = (0.12, 0.78 , 0.12), opacity = 1)

#     #L4_actor = trimesh_vtk.point_cloud_actor(L4_loc, size = 6000, color = ([0, 1, 0],[0, 1, 0],[0, 1, 0]), opacity = 1)
#     #cyan
#     L5it_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("5P-IT")]
#     L5it_loc = np.vstack(L5it_syn['ctr_pt_position'])*[4,4,40]
#     L5it_actor = trimesh_vtk.point_cloud_actor(L5it_loc, size = 6000, color = (0, 1, 1), opacity = 1)
# #     L5it_actor = trimesh_vtk.point_cloud_actor(L5it_loc, size = 6000, color = (0.30, 0.84, 0.88), opacity = 1)

#     #yellow
#     L5et_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("5P-ET")]
#     L5et_loc = np.vstack(L5et_syn['ctr_pt_position'])*[4,4,40]
#     L5et_actor = trimesh_vtk.point_cloud_actor(L5et_loc, size = 6000, color = (1, 1, 0), opacity = 1)
# #     L5et_actor = trimesh_vtk.point_cloud_actor(L5et_loc, size = 6000, color = (0.88, 0.82 , 0.30), opacity = 1)
#     #L5pt_actor = trimesh_vtk.point_cloud_actor(L5pt_loc, size = 4000, color = (0.8, 0.8, 0), opacity = 0.9)
    
    
#     #magenta
#     L6_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("6P")]
#     L6_loc = np.vstack(L6_syn['ctr_pt_position'])*[4,4,40]
#     L6_actor = trimesh_vtk.point_cloud_actor(L6_loc, size = 6000, color = (1, 0, 1), opacity = 1)
# #     L6_actor = trimesh_vtk.point_cloud_actor(L6_loc, size = 6000, color = (0.88, 0.3, 0.88), opacity = 1)

    
#     # #light pink
#     INH_syn = dfm_i.loc[dfm_i['cell_type'].str.contains("INH")]
#     INH_loc = np.vstack(INH_syn['ctr_pt_position'])*[4,4,40]
# #     INH_actor = trimesh_vtk.point_cloud_actor(INH_loc, size = 6000, color = (1, 0.8, 1), opacity = 1)
#     INH_actor = trimesh_vtk.point_cloud_actor(INH_loc, size = 6000, color = (0.92, 0.64, 0.80), opacity = 1)


    if i == 864691135467660940:
        pia_loc1 = np.array([[52475, 87307, 21083],[52475, 108587, 21083],[52475, 164808, 21083],[52475, 179701, 21083],[52475, 221701, 21083],[52475, 254258, 21083],[52475, 261402, 21083], [52000, 270000, 21083]])*[4,4,40]
        pia_loc2 = np.array([[334807, 87307, 21083],[334807, 108587, 21083],[334807, 164808, 21083],[334807, 179701, 21083],[334807, 221701, 21083],[334807, 254258, 21083],[334807, 261402, 21083], [77000, 270000, 21083]])*[4,4,40]
    
#         endpts1 = np.vstack([[177292, 173536, 20742],[176253, 162625, 20550]])*[4,4,40]
#         endpts2 = np.vstack([[176356, 159174, 20642],[175074, 144572, 20628]])*[4,4,40]
#         endpts3 = np.vstack([[174931, 143463, 20634],[173755, 127828, 20509]])*[4,4,40]
#         endpts4 = np.vstack([[173822, 126146, 20516],[173527, 113650, 20633]])*[4,4,40]
#         endpts5 = np.vstack([[173539, 113410, 20645], [173651, 108373, 20714]])*[4,4,40]
#         endpts6 = np.vstack([[173477, 107846, 20701], [177720, 99206, 20929]])*[4,4,40]

#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6]
        
#     elif i == 864691135544588584:
#         pia_loc1 = np.array([[52475, 86761, 18916],[52475, 106904, 18916],[52475, 167439, 18916],[52475, 181547, 18916],[52475, 221096, 18916],[52475, 254112, 18916],[52475, 260462, 18916], [52000, 270000, 18916]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 86761, 18916],[334807, 106904, 18916],[334807, 167439, 18916],[334807, 181547, 18916],[334807, 221096, 18916],[334807, 254112, 18916],[334807, 260462, 18916], [77000, 270000, 18916]])*[4,4,40]
    
#         endpts1 = np.vstack([[127859, 173510, 18894],[127272, 168274, 18794]])*[4,4,40]
           
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         listact = [mye_act1]
        
#     elif i == 864691135699487522:
#         pia_loc1 = np.array([[52475, 77294, 24049],[52475, 100883, 24049],[52475, 167755, 24049],[52475, 182636, 24049],[52475, 217662, 24049],[52475, 251525, 24049],[52475, 259991, 24049], [52000, 270000, 24049]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 77294, 24049],[334807, 100883, 24049],[334807, 167755, 24049],[334807, 182636, 24049],[334807, 217662, 24049],[334807, 251525, 24049],[334807, 259991, 24049], [77000, 270000, 24049]])*[4,4,40]
    
#         endpts1 = np.vstack([[234909, 174893, 23999],[234090, 167553, 24032]])*[4,4,40]
#         endpts2 = np.vstack([[234092, 167013, 24034],[233772, 160832, 24015]])*[4,4,40]
#         endpts3 = np.vstack([[233693, 160585, 24011],[233581, 156818, 23969]])*[4,4,40]
#         endpts4 = np.vstack([[232672, 153635, 23925],[231129, 147481, 23879]])*[4,4,40]
#         endpts5 = np.vstack([[230322, 146320, 23815], [229017, 140043, 23645]])*[4,4,40]
#         endpts6 = np.vstack([[234645, 153458, 23209], [239273, 155517, 22300]])*[4,4,40]

#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4, mye_act5, mye_act6]
        
#     elif i == 864691135925834510:
#         pia_loc1 = np.array([[52475, 85380, 21390],[52475, 112747, 21390],[52475, 170290, 21390],[52475, 184731, 21390],[52475, 231706, 21390],[52475, 257749, 21390],[52475, 262607, 21390], [52000, 270000, 21390]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 85380, 21390],[334807, 112747, 21390],[334807, 170290, 21390],[334807, 184731, 21390],[334807, 231706, 21390],[334807, 257749, 21390],[334807, 262607, 21390], [77000, 270000, 21390]])*[4,4,40]

#         endpts1 = np.vstack([[188324, 186481, 21490],[187536, 191760, 21497]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         listact = [mye_act1]
                
#     elif i == 864691136618564493:
#         pia_loc1 = np.array([[52475, 77792, 24421],[52475, 100421, 24421],[52475, 163000, 24421],[52475, 178419, 24421],[52475, 221881, 24421],[52475, 255286, 24421],[52475, 262272, 24421], [52000, 270000, 24421]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 77792, 24421],[334807, 100421, 24421],[334807, 163000, 24421],[334807, 178419, 24421],[334807, 221881, 24421],[334807, 255286, 24421],[334807, 262272, 24421], [77000, 270000, 24421]])*[4,4,40]

#         endpts1 = np.vstack([[197781, 109956, 24497],[195060, 105843, 24487]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         listact = [mye_act1]
        
#     elif i ==864691136116457636:
#         pia_loc1 = np.array([[52475, 77792, 24421],[52475, 100421, 24421],[52475, 163000, 24421],[52475, 178419, 24421],[52475, 221881, 24421],[52475, 255286, 24421],[52475, 262272, 24421], [52000, 270000, 24421]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 77792, 24421],[334807, 100421, 24421],[334807, 163000, 24421],[334807, 178419, 24421],[334807, 221881, 24421],[334807, 255286, 24421],[334807, 262272, 24421], [77000, 270000, 24421]])*[4,4,40]
        
#         listact = []
        
#     elif i == 864691136118647832:
#         pia_loc1 = np.array([[52475, 79998, 22882],[52475, 100589, 22882],[52475, 164116, 22882],[52475, 178364, 22882],[52475, 218149, 22882],[52475, 251773, 22882],[52475, 262562, 22882], [52000, 270000, 22882]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 79998, 22882],[334807, 100589, 22882],[334807, 164116, 22882],[334807, 178364, 22882],[334807, 218149, 22882],[334807, 251773, 22882],[334807, 262562, 22882], [77000, 270000, 22882]])*[4,4,40]
        
#         endpts1 = np.vstack([[218268, 167477, 23340],[219674, 156325, 23502]])*[4,4,40]
#         endpts2 = np.vstack([[219639, 155784, 23511],[217730, 146654, 23482]])*[4,4,40]
#         endpts3 = np.vstack([[217572, 146390, 23462],[215209, 136055, 23495]])*[4,4,40]
#         endpts4 = np.vstack([[214945, 134684, 23529],[215926, 118708, 23617]])*[4,4,40]
#         endpts5 = np.vstack([[215830, 117705, 23606], [215094, 99455, 23631]])*[4,4,40]
#         endpts6 = np.vstack([[214740, 97768, 23457], [211683, 94724, 22953]])*[4,4,40]

#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6]
        
        
#     elif i == 864691135058985115:
#         pia_loc1 = np.array([[52475, 79998, 22882],[52475, 100589, 22882],[52475, 164116, 22882],[52475, 178364, 22882],[52475, 218149, 22882],[52475, 251773, 22882],[52475, 262562, 22882], [52000, 270000, 22882]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 79998, 22882],[334807, 100589, 22882],[334807, 164116, 22882],[334807, 178364, 22882],[334807, 218149, 22882],[334807, 251773, 22882],[334807, 262562, 22882], [77000, 270000, 22882]])*[4,4,40]

#         endpts1 = np.vstack([[169619, 187320, 22543], [169156, 194422, 22606]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         listact = [mye_act1]
        
    elif i == 864691135013417622:
        pia_loc1 = np.array([[52475, 86538, 19168],[52475, 107942, 19168],[52475, 163621, 19168],[52475, 181885, 19168],[52475, 220009, 19168],[52475, 250970, 19168],[52475, 259409, 19168], [52000, 270000, 19168]])*[4,4,40]
        pia_loc2 = np.array([[334807, 86538, 19168],[334807, 107942, 19168],[334807, 163621, 19168],[334807, 181885, 19168],[334807, 220009, 19168],[334807, 250970, 19168],[334807, 259409, 19168], [77000, 270000, 19168]])*[4,4,40]
        
#         endpts1 = np.vstack([[228584, 175614, 18640],[229170, 180054, 18802]])*[4,4,40] 
#         endpts2 = np.vstack([[226374, 167113, 18487],[228489, 175371, 18628]])*[4,4,40] 
#         endpts3 = np.vstack([[225509, 160085, 18494],[226325, 166686, 18483]])*[4,4,40] 
#         endpts4 = np.vstack([[225117, 154849, 18577],[225592, 159514, 18497]])*[4,4,40] 
#         endpts5 = np.vstack([[223307, 143190, 18443],[224525, 152803, 18525]])*[4,4,40] 
#         endpts6 = np.vstack([[221958, 135431, 18294],[223031, 142574, 18421]])*[4,4,40] 
#         endpts7 = np.vstack([[222721, 126103, 18216],[222067, 134840, 18286]])*[4,4,40] 
#         endpts8 = np.vstack([[221328, 111104, 18227],[222871, 125458, 18219]])*[4,4,40] 
#         endpts9 = np.vstack([[220689, 106030, 18394],[221214, 110382, 18232]])*[4,4,40] 
#         endpts10 = np.vstack([[221482, 99723, 18645],[220083, 103447, 18394]])*[4,4,40] 
#         endpts11 = np.vstack([[212886, 98570, 17493],[220220, 103576, 18384]])*[4,4,40] 
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         mye_filt7 = mesh_filters.filter_two_point_distance(mesh, endpts7,1500)
#         mye_mesh7 = mesh.apply_mask(mye_filt7)
#         mye_act7 = trimesh_vtk.mesh_actor(mye_mesh7, color = (1,0,1), opacity = 1)
#         mye_act7.GetProperty().SetRepresentationToWireframe()
#         mye_act7.GetProperty().SetLineWidth(20)
        
#         mye_filt8 = mesh_filters.filter_two_point_distance(mesh, endpts8,1500)
#         mye_mesh8 = mesh.apply_mask(mye_filt8)
#         mye_act8 = trimesh_vtk.mesh_actor(mye_mesh8, color = (1,0,1), opacity = 1)
#         mye_act8.GetProperty().SetRepresentationToWireframe()
#         mye_act8.GetProperty().SetLineWidth(20)

#         mye_filt9 = mesh_filters.filter_two_point_distance(mesh, endpts9,1500)
#         mye_mesh9 = mesh.apply_mask(mye_filt9)
#         mye_act9 = trimesh_vtk.mesh_actor(mye_mesh9, color = (1,0,1), opacity = 1)
#         mye_act9.GetProperty().SetRepresentationToWireframe()
#         mye_act9.GetProperty().SetLineWidth(20)

#         mye_filt10 = mesh_filters.filter_two_point_distance(mesh, endpts10,1500)
#         mye_mesh10 = mesh.apply_mask(mye_filt10)
#         mye_act10 = trimesh_vtk.mesh_actor(mye_mesh10, color = (1,0,1), opacity = 1)
#         mye_act10.GetProperty().SetRepresentationToWireframe()
#         mye_act10.GetProperty().SetLineWidth(20)
        
#         mye_filt11 = mesh_filters.filter_two_point_distance(mesh, endpts11,1500)
#         mye_mesh11 = mesh.apply_mask(mye_filt11)
#         mye_act11 = trimesh_vtk.mesh_actor(mye_mesh11, color = (1,0,1), opacity = 1)
#         mye_act11.GetProperty().SetRepresentationToWireframe()
#         mye_act11.GetProperty().SetLineWidth(20)        
        
#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6, mye_act7, mye_act8 ,mye_act9, mye_act10, mye_act11]

#     elif i ==864691135118298333:
#         pia_loc1 = np.array([[52475, 86538, 19168],[52475, 107942, 19168],[52475, 163621, 19168],[52475, 181885, 19168],[52475, 220009, 19168],[52475, 250970, 19168],[52475, 259409, 19168], [52000, 270000, 19168]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 86538, 19168],[334807, 107942, 19168],[334807, 163621, 19168],[334807, 181885, 19168],[334807, 220009, 19168],[334807, 250970, 19168],[334807, 259409, 19168], [77000, 270000, 19168]])*[4,4,40]

        
#         endpts1 = np.vstack([[237530, 183408, 18704],[236946, 187289, 18784]])*[4,4,40]
#         endpts2 = np.vstack([[237294, 178657, 18595], [237645, 183204, 18694]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)
        
#         listact = [mye_act1, mye_act2]
        
#     elif i == 864691135374222153:
#         pia_loc1 = np.array([[52475, 83959, 21466],[52475, 104622, 21466],[52475, 162280, 21466],[52475, 182474, 21466],[52475, 227371, 21466],[52475, 252297, 21466], [52475, 259229, 21466], [52000, 270000, 21466]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 83959, 21466],[334807, 104622, 21466],[334807, 162280, 21466],[334807, 182474, 21466],[334807, 227371, 21466],[334807, 252297, 21466], [334807, 259229, 21466], [77000, 270000, 21466]])*[4,4,40]

#         endpts1 = np.vstack([[171918, 202347, 21292],[173227, 207498, 21400]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)
        
#         listact = [mye_act1]
        
#     elif i == 864691135577202181:
#         pia_loc1 = np.array([[52475, 88085, 18273],[52475, 107295, 18273],[52475, 160909, 18273],[52475, 172473, 18273],[52475, 215166, 18273],[52475, 248949, 18273],[52475, 257028, 18273], [52000, 270000, 18273]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 88085, 18273],[334807, 107295, 18273],[334807, 160909, 18273],[334807, 172473, 18273],[334807, 215166, 18273],[334807, 248949, 18273],[334807, 257028, 18273], [77000, 270000, 18273]])*[4,4,40]

        
#         endpts1 = np.vstack([[151620, 174970, 18354],[152422, 169262, 18464]])*[4,4,40]
#         endpts2 = np.vstack([[152990, 158753, 18529],[152043, 152119, 18495]])*[4,4,40]
#         endpts3 = np.vstack([[151948, 151816, 18495],[151884, 147860, 18491]])*[4,4,40]
#         endpts4 = np.vstack([[151977, 147490, 18507],[150243, 138016, 18524]])*[4,4,40]
#         endpts5 = np.vstack([[150181, 137211, 18520], [150880, 128658, 18585]])*[4,4,40]
#         endpts6 = np.vstack([[150611, 124059, 18615], [150797, 112465, 18713]])*[4,4,40]
#         endpts7 = np.vstack([[152552, 168565, 18486], [152608, 164043, 18563]])*[4,4,40]
#         endpts8 = np.vstack([[152662, 163803, 18562], [152575, 159722, 18522]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         mye_filt7 = mesh_filters.filter_two_point_distance(mesh, endpts7,1500)
#         mye_mesh7 = mesh.apply_mask(mye_filt7)
#         mye_act7 = trimesh_vtk.mesh_actor(mye_mesh7, color = (1,0,1), opacity = 1)
#         mye_act7.GetProperty().SetRepresentationToWireframe()
#         mye_act7.GetProperty().SetLineWidth(20)
        
#         mye_filt8 = mesh_filters.filter_two_point_distance(mesh, endpts8,1500)
#         mye_mesh8 = mesh.apply_mask(mye_filt8)
#         mye_act8 = trimesh_vtk.mesh_actor(mye_mesh8, color = (1,0,1), opacity = 1)
#         mye_act8.GetProperty().SetRepresentationToWireframe()
#         mye_act8.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6, mye_act7, mye_act8]
        
#     elif i== 864691135988665856:
#         pia_loc1 = np.array([[52475, 87386, 20750],[52475, 108199, 20750],[52475, 165250, 20750],[52475, 177247, 20750],[52475, 228186, 20750],[52475, 255174, 20750],[52475, 261438, 20750], [52000, 270000, 20750]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 87386, 20750],[334807, 108199, 20750],[334807, 165250, 20750],[334807, 177247, 20750],[334807, 228186, 20750],[334807, 255174, 20750],[334807, 261438, 20750], [77000, 270000, 20750]])*[4,4,40]

#         endpts1 = np.vstack([[172215, 194155, 20848],[ 172697, 199094, 21027]])*[4,4,40]   
#         endpts2 = np.vstack([[165299, 151400, 20436],[166798, 160721, 20415]])*[4,4,40]   
#         endpts3 = np.vstack([[164378, 143503, 20363],[165199, 151135, 20436]])*[4,4,40]   
#         endpts4 = np.vstack([[163502, 132048, 20353],[163881, 137089, 20360]])*[4,4,40]

#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4]

#     elif i == 864691135754152141:
#         pia_loc1 = np.array([[52475, 84030, 21929],[52475, 104743, 21929],[52475, 160158, 21929],[52475, 177539, 21929],[52475, 220917, 21929],[52475, 248837, 21929],[52475, 258132, 21929], [52000, 270000, 21929]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 84030, 21929],[334807, 104743, 21929],[334807, 160158, 21929],[334807, 177539, 21929],[334807, 220917, 21929],[334807, 248837, 21929],[334807, 258132, 21929], [77000, 270000, 21929]])*[4,4,40]

#         endpts1 = np.vstack([[173767, 211059, 22000],[171920, 204931, 22077]])*[4,4,40]
#         endpts2 = np.vstack([[171817, 201843, 22002],[169937, 194647, 21925]])*[4,4,40]
#         endpts3 = np.vstack([[170139, 192128, 21962],[168688, 185687, 21948]])*[4,4,40]
#         endpts4 = np.vstack([[168588, 185264, 21934],[168458, 184823, 21905]])*[4,4,40]
#         endpts5 = np.vstack([[168903, 174216, 21790], [168890, 161502, 21794]])*[4,4,40]
#         endpts6 = np.vstack([[168748, 160298, 21752], [166482, 148836, 21711]])*[4,4,40]

        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)
        
#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6]

        
#     elif i == 864691135341516741:
#         pia_loc1 = np.array([[52475, 75842, 24555],[52475, 101362, 24555],[52475, 160420, 24555],[52475, 178604, 24555],[52475, 220257, 24555],[52475, 250778, 24555],[52475, 259564, 24555], [52000, 270000, 24555]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 75842, 24555],[334807, 101362, 24555],[334807, 160420, 24555],[334807, 178604, 24555],[334807, 220257, 24555],[334807, 250778, 24555],[334807, 259564, 24555], [77000, 270000, 24555]])*[4,4,40]
  
#         endpts1 = np.vstack([[243267, 194742, 24568],[243091, 198112, 24571]])*[4,4,40]
#         endpts2 = np.vstack([[241927, 160703, 24096],[242201, 172800, 24323]])*[4,4,40]
#         endpts3 = np.vstack([[240996, 147247, 24012],[241816, 157490, 24056]])*[4,4,40]
#         endpts4 = np.vstack([[239880, 127692, 23997],[240014, 135445, 24035]])*[4,4,40]
#         endpts5 = np.vstack([[240113, 122591, 24023],[239918, 127279, 24002]])*[4,4,40]
#         endpts6 = np.vstack([[241153, 115195, 23952],[240362, 120418, 23987]])*[4,4,40]
#         endpts7 = np.vstack([[243714, 100533, 23453],[242023, 107697, 23743]])*[4,4,40]
        
#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         mye_filt7 = mesh_filters.filter_two_point_distance(mesh, endpts7,1500)
#         mye_mesh7 = mesh.apply_mask(mye_filt7)
#         mye_act7 = trimesh_vtk.mesh_actor(mye_mesh7, color = (1,0,1), opacity = 1)
#         mye_act7.GetProperty().SetRepresentationToWireframe()
#         mye_act7.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6, mye_act7]


#     elif i == 864691135785592004:
#         pia_loc1 = np.array([[52475, 87483, 18766],[52475, 109523, 18766],[52475, 167700, 18766],[52475, 183154, 18766],[52475, 220056, 18766],[52475, 253861, 18766],[52475, 259886, 18766], [52000, 270000, 18916]])*[4,4,40]
#         pia_loc2 = np.array([[334807, 87483, 18766],[334807, 109523, 18766],[334807, 167700, 18766],[334807, 183154, 18766],[334807, 220056, 18766],[334807, 253861, 18766],[334807, 259886, 18766], [77000, 270000, 18916]])*[4,4,40]

#         endpts1 = np.vstack([[111009, 160956, 18472],[110145, 152451, 18512]])*[4,4,40]
#         endpts2 = np.vstack([[109016, 146494, 18545],[107964, 139044, 18534]])*[4,4,40]
#         endpts3 = np.vstack([[106165, 130905, 18510],[105652, 124108, 18476]])*[4,4,40]
#         endpts4 = np.vstack([[114253, 185890, 18530],[113680, 182842, 18448]])*[4,4,40]
#         endpts5 = np.vstack([[113653, 191595, 18736], [114350, 186232, 18536]])*[4,4,40]
#         endpts6 = np.vstack([[113045, 178142, 18393], [112586, 173837, 18475]])*[4,4,40]
#         endpts7 = np.vstack([[112402, 173289, 18476], [111162, 168345, 18422]])*[4,4,40]

#         #scale bar
#         # scale1 = np.array([[52000, 270000, 18273]])*[4,4,40]
#         # scale2 = np.array([ [77000, 270000, 18273]])*[4,4,40]

#         mye_filt1 = mesh_filters.filter_two_point_distance(mesh, endpts1,1500)
#         mye_mesh1 = mesh.apply_mask(mye_filt1)
#         mye_act1 = trimesh_vtk.mesh_actor(mye_mesh1, color = (1,0,1), opacity = 1)
#         mye_act1.GetProperty().SetRepresentationToWireframe()
#         mye_act1.GetProperty().SetLineWidth(20)

#         mye_filt2 = mesh_filters.filter_two_point_distance(mesh, endpts2,1500)
#         mye_mesh2 = mesh.apply_mask(mye_filt2)
#         mye_act2 = trimesh_vtk.mesh_actor(mye_mesh2, color = (1,0,1), opacity = 1)
#         mye_act2.GetProperty().SetRepresentationToWireframe()
#         mye_act2.GetProperty().SetLineWidth(20)

#         mye_filt3 = mesh_filters.filter_two_point_distance(mesh, endpts3,1500)
#         mye_mesh3 = mesh.apply_mask(mye_filt3)
#         mye_act3 = trimesh_vtk.mesh_actor(mye_mesh3, color = (1,0,1), opacity = 1)
#         mye_act3.GetProperty().SetRepresentationToWireframe()
#         mye_act3.GetProperty().SetLineWidth(20)

#         mye_filt4 = mesh_filters.filter_two_point_distance(mesh, endpts4,1500)
#         mye_mesh4 = mesh.apply_mask(mye_filt4)
#         mye_act4 = trimesh_vtk.mesh_actor(mye_mesh4, color = (1,0,1), opacity = 1)
#         mye_act4.GetProperty().SetRepresentationToWireframe()
#         mye_act4.GetProperty().SetLineWidth(20)

#         mye_filt5 = mesh_filters.filter_two_point_distance(mesh, endpts5,1500)
#         mye_mesh5 = mesh.apply_mask(mye_filt5)
#         mye_act5 = trimesh_vtk.mesh_actor(mye_mesh5, color = (1,0,1), opacity = 1)
#         mye_act5.GetProperty().SetRepresentationToWireframe()
#         mye_act5.GetProperty().SetLineWidth(20)

#         mye_filt6 = mesh_filters.filter_two_point_distance(mesh, endpts6,1500)
#         mye_mesh6 = mesh.apply_mask(mye_filt6)
#         mye_act6 = trimesh_vtk.mesh_actor(mye_mesh6, color = (1,0,1), opacity = 1)
#         mye_act6.GetProperty().SetRepresentationToWireframe()
#         mye_act6.GetProperty().SetLineWidth(20)

#         mye_filt7 = mesh_filters.filter_two_point_distance(mesh, endpts7,1500)
#         mye_mesh7 = mesh.apply_mask(mye_filt7)
#         mye_act7 = trimesh_vtk.mesh_actor(mye_mesh7, color = (1,0,1), opacity = 1)
#         mye_act7.GetProperty().SetRepresentationToWireframe()
#         mye_act7.GetProperty().SetLineWidth(20)

#         listact = [mye_act1, mye_act2, mye_act3, mye_act4 ,mye_act5, mye_act6, mye_act7]

    la = trimesh_vtk.linked_point_actor(pia_loc1, pia_loc2, inds_a = None, inds_b = None, line_width = 3, color = (0.8,0.8,0.8), opacity = 0.7)    
    camera = vtk.vtkCamera()
    view = camera.ParallelProjectionOn()
    focal2 = np.array([183104, 177901, 22605])*[4,4,40]
#     orient2 = np.array([0,0,0,1])
    orient2 = np.array([0,0.2,0,1])

#     camera_2 = trimesh_vtk.camera_from_quat(focal2, orient2,camera_distance=1600000, ngl_correct=True)
#     listact.append(prema)
#     listact.append(sa)
#     listact.append(la)
# #     camera3 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0, 0, -1], up_vector = [0, -1, 0])                        
# #     camera_start = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0, 0, 1], up_vector = [0, -1, 0])
# #     camera_15 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.259, 0, 0.97], up_vector = [0, -1, 0])
# #     camera_30 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.499, 0, 0.86], up_vector = [0, -1, 0])
# #     camera_45 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.707, 0, 0.707], up_vector = [0, -1, 0])
# #     camera_60 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.86, 0, 0.499], up_vector = [0, -1, 0])
    
# #     camera_75 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.97, 0,0.259], up_vector = [0, -1, 0])
# #     camera_90 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[1, 0, 0], up_vector = [0, -1, 0])
# #     camera_105 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.97, 0, -0.259], up_vector = [0, -1, 0])
# #     camera_120 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.867, 0, -0.499], up_vector = [0, -1, 0])
    
# #     camera_135 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.707, 0, -0.707], up_vector = [0, -1, 0])
# #     camera_150 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.499, 0, -0.867], up_vector = [0, -1, 0])
# #     camera_165 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0.259, 0, -0.97], up_vector = [0, -1, 0])
    camera_180 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[0, 0, -1], up_vector = [0, -1, 0])                        

# #     camera_195 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.259, 0, -0.97], up_vector = [0, -1, 0])
# #     camera_210 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.499, 0, -0.86], up_vector = [0, -1, 0])
# #     camera_225 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.707, 0, -0.707], up_vector = [0, -1, 0])
# #     camera_240 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.86, 0, -0.499], up_vector = [0, -1, 0])
    
# #     camera_255 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.97, 0,-0.259], up_vector = [0, -1, 0])     
# #     camera_270 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-1, 0, 0], up_vector = [0, -1, 0])
# #     camera_285 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.97, 0, 0.259], up_vector = [0, -1, 0])
# #     camera_300 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.867, 0, 0.499], up_vector = [0, -1, 0])
    
# #     camera_315 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.707, 0, 0.707], up_vector = [0, -1, 0])
# #     camera_330 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.499, 0, 0.867], up_vector = [0, -1, 0])
# #     camera_345 = trimesh_vtk.oriented_camera([focal2], backoff=2000,backoff_vector=[-0.259, 0, 0.97], up_vector = [0, -1, 0])

# #     trimesh_vtk.render_actors(listact,
# #                               filename = "{}_outputsyncyan_myemagenta_layers_scale.png".format(i), do_save = True, camera = view)
# #     trimesh_vtk.render_actors([la, prema, L23_actor, L4_actor, L5it_actor, L5et_actor, L6_actor],
#     trimesh_vtk.render()
    trimesh_vtk.render_actors([la, prema],
                            filename = "{}_prema_layers_scale_61_180.png".format(i), back_color=(1, 1, 1),
                              do_save = False, camera = camera_180)



864691135013417622


In [ ]:
# pre_id_mc = [864691135988665856]

dfm_i = dfm_somas.loc[dfm_somas['pre_pt_root_id'] == 864691135988665856]
mesh = mm.mesh(seg_id = 864691135988665856)
prema = trimesh_vtk.mesh_actor(mesh, color =(0.3,0.3,0.3), opacity= 1)
prema.GetProperty().SetRepresentationToWireframe()
prema.GetProperty().SetLineWidth(1)

camera = vtk.vtkCamera()
view = camera.ParallelProjectionOn()
trimesh_vtk.render_actors_360([la],'movie_370', 270, camera_start = view, do_save = True)